##Importar Librerias

In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import sys 

###Librerias para DL (Deep Learning)

In [2]:
import tensorflow as tf

In [3]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.7.0
Eager mode:  True
GPU is available


##Activando la GPU

In [4]:
tf.device('/cpu:0')

##Descarga y preprocesado de los datos

In [5]:
fileDL= tf.keras.utils.get_file('cuentos2.txt','https://raw.githubusercontent.com/michelBolivar/Deep_learning/main/Corte2/cuento/cuentos2.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')
texto = texto.lower()

65536/55371 [===================================] - 0s 0us/step


###Entendiendo el texto

In [6]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:54195 caracteres
el texto esta compuesto de estos :55 caracteres
['\n', ' ', '!', "'", '(', ')', ',', '-', '.', '1', '2', '3', '4', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'à', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '—']


###Pasar el texto a números

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: 'el pantano de la luna denys barry se ha esfumado en alguna parte, en alguna región espantosa y remot'
array([20, 27,  1, 31, 16, 29, 35, 16, 29, 30,  1, 19, 20,  1, 27, 16,  1,
       27, 36, 29, 16,  1, 19, 20, 29, 40, 34,  1, 17, 16, 33, 33, 40,  1,
       34, 20,  1, 23, 16,  1, 20, 34, 21, 36, 28, 16, 19, 30,  1, 20, 29,
        1, 16, 27, 22, 36, 29, 16,  1, 31, 16, 33, 35, 20,  6,  1, 20, 29,
        1, 16, 27, 22, 36, 29, 16,  1, 33, 20, 22, 24, 51, 29,  1, 20, 34,
       31, 16, 29, 35, 30, 34, 16,  1, 40,  1, 33, 20, 28, 30, 35])


###Preparar los datos para ser usados en la RNN

In [8]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=70
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'el pantano de la luna denys barry se ha esfumado en alguna parte, en al'
'guna región espantosa y remota de la que nada sé. estaba con él la últi'
'ma noche que pasó entre los hombres, y escuché sus gritos cuando el ser'
' lo atacó; pero, ni todos los campesinos y policías del condado de meat'
'h pudieron encontrarlo, ni a él ni a los otros, aunque los buscaron por'
' todas partes. y ahora me estremezco cuando oigo croar a las ranas en l'
'os pantanos o veo la luna en lugares solitarios. había intimado con den'
'ys barry en estados unidos, donde éste se había hecho rico, y lo felici'
'té cuando recompró el viejo castillo junto al pantano, en el somnolient'
'o kilderry. de kilderry procedía su padre, y allí era donde quería disf'


####Separar los datos en agrupamientos (batches)

In [9]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  'el pantano de la luna denys barry se ha esfumado en alguna parte, en a'
Target data:  'l pantano de la luna denys barry se ha esfumado en alguna parte, en al'


In [10]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64 #Registro de fila
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

<MapDataset shapes: ((70,), (70,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 70), (64, 70)), types: (tf.int64, tf.int64)>


##Construcción del modelo RNN

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  #Se utilizan tres capas 
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=build_model(vocab_size=vocab_size,
                  embedding_dim=embedding_dim,
                  rnn_units=rnn_units,
                  batch_size=BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           14080     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 55)            56375     
                                                                 
Total params: 5,317,431
Trainable params: 5,317,431
Non-trainable params: 0
_________________________________________________________________


##Entrenando la RNN

In [12]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

#En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
#con los argumentos por defecto del optimizador Adam.  
model.compile(optimizer='adam',loss=loss)

###Creando chekpoints

In [13]:
checkpoint_dir='/content/drive/MyDrive/Deep/checkpoint'
checkpoint_prefix= os.path.join(checkpoint_dir,"ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

###Entrenando

In [14]:
EPOCHS=500
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/500
11/11 [==============================] - 8s 159ms/step - loss: 3.2639
Epoch 2/500
11/11 [==============================] - 2s 207ms/step - loss: 2.9036
Epoch 3/500
11/11 [==============================] - 2s 148ms/step - loss: 2.8391
Epoch 4/500
11/11 [==============================] - 2s 146ms/step - loss: 2.7039
Epoch 5/500
11/11 [==============================] - 2s 151ms/step - loss: 2.4999
Epoch 6/500
11/11 [==============================] - 2s 150ms/step - loss: 2.3616
Epoch 7/500
11/11 [==============================] - 2s 160ms/step - loss: 2.2943
Epoch 8/500
11/11 [==============================] - 5s 485ms/step - loss: 2.2328
Epoch 9/500
11/11 [==============================] - 2s 173ms/step - loss: 2.1864
Epoch 10/500
11/11 [==============================] - 2s 156ms/step - loss: 2.1521
Epoch 11/500
11/11 [==============================] - 4s 368ms/step - loss: 2.1105
Epoch 12/500
11/11 [==============================] - 2s 181ms/step - loss: 2.0734
Epoch 13/500


##Generando texto nuevo usando la RNN

In [15]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [16]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.5  
  #entre más alta la temperatura más creatividad al modelo, pero tambien
  #más errores ortograficos.
  model.reset_states()
  #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

###Generando texto

In [17]:
print(generate_text(model, start_string=u"merodeando sobre las aguas"))

merodeando sobre las aguas que no la fameniza de tras estansinados y los primeros de los primeros pero tudo perseguido espantosa y cuando los alegres como no había dermido y invación con falaba teciliero que de no pen alguna tande una maldición que, en adelante, afligiría a la casa de c. «nunca sea que un noble de tumento a los condes de fiente en el brejo de la colina se encontró sumido en la base de tambría y el elixir de la eterna juventud, y tenía fama de ducho con el castillo de la noche. el conde murió sin decir pa


##Exportando modelo

In [19]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/Deep/Modelo/modelRNN_cuentos2_pesos.hdf5")
model.save('modelRNN_cuentos2.h5')
print("modelo salvado en disco")

modelo salvado en disco
